In [ ]:
#@title Connect to the Stability API

import cv2
import math
import numpy as np
import os
import random
import shutil
import subprocess
import sys

from pathlib import Path
from PIL import Image

try:
    import gradio as gr
except:
    subprocess.run(['pip', 'install', 'gradio'])
    import gradio as gr

if os.path.exists("../src/stability_sdk"):    
    # use local SDK src
    sys.path.append("../src")
else:
    # install Stability SDK for Python
    path = Path('stability-sdk')
    if path.exists():
        shutil.rmtree(path)
        subprocess.run(['pip', 'uninstall', '-y', 'stability-sdk'])
    result = subprocess.run(['git', 'clone', '-b', 'pharma.inpaint', '--recurse-submodules', 'https://github.com/Stability-AI/stability-sdk'], stdout=subprocess.PIPE).stdout.decode('utf-8')
    print(result)
    result = subprocess.run(['pip', 'install', './stability-sdk'], stdout=subprocess.PIPE).stdout.decode('utf-8')
    print(result)

import stability_sdk.interfaces.gooseai.generation.generation_pb2 as generation

from stability_sdk import client
from stability_sdk import utils

# GRPC endpoint and engines
GRPC_HOST = "grpc.stability.ai:443" #@param {type:"string"}
API_KEY = "" #@param {type:"string"}

# Connect to Stability API
channel = client.open_channel(GRPC_HOST, api_key=API_KEY)
api = client.Api(channel=channel)

In [ ]:
#@title Inpainting Gradio UI

def shuffle_for_inpaint(image: np.ndarray, mask: np.ndarray, rand_radius: float=5) -> np.ndarray:
    image = image.copy()
    w, h = image.shape[1], image.shape[0]
    for y in range(0, h):
        for x in range(0, w):
            if mask[y, x][0] == 255:
                continue
            rand_x = max(0, min(w-1, random.randint(round(x-rand_radius), round(x+rand_radius))))
            rand_y = max(0, min(h-1, random.randint(round(y-rand_radius), round(y+rand_radius))))
            t = image[rand_y, rand_x]
            if mask[rand_y, rand_x][0] == 0:
                image[rand_y, rand_x] = image[y, x]
            image[y, x] = t

    # blur the shuffled pixels (eliminates noisy border between masked and unmasked)
    ks = math.ceil(rand_radius)*2
    gr = min(7, int(rand_radius//2*2+1))
    eroded_mask = cv2.erode(mask, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (ks, ks)), iterations=1)
    blurred = cv2.GaussianBlur(image, (gr, gr), 0)
    image = utils.image_mix(blurred, image, eroded_mask)

    return image

def inpaint(
    model: str, prompt: str, seed: int, cfg_scale: float, init_strength: float, 
    mask_fixup: bool, mask_init: str, image: Image.Image, mask: Image.Image, 
    shuffle_radius: float, mask_erode: int, mask_blur: int, mode: str
) -> Image.Image:
    image = utils.pil_to_cv2(image)
    mask = utils.pil_to_cv2(mask)
    w, h = image.shape[1], image.shape[0]

    if mask_init == "original":
        masked_area_init = generation.MASKED_AREA_INIT_ORIGINAL
    elif mask_init == "random":
        masked_area_init = generation.MASKED_AREA_INIT_RANDOM
    elif mask_init == "zero":
        masked_area_init = generation.MASKED_AREA_INIT_ZERO

    # optionally shuffle the placeholder pixels
    if shuffle_radius > 0:        
        image = shuffle_for_inpaint(image, mask, shuffle_radius)

    # optionally erode the inpainting mask
    if mask_erode > 0:
        ks = mask_erode*2 + 1
        mask = cv2.erode(mask, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (ks, ks)), iterations=1)

    # optionally blur the inpainting mask
    if mask_blur > 0:
        ks = mask_blur*2 + 1
        mask = cv2.GaussianBlur(mask, (ks, ks), 0)

    if mode == 'result':
        if model == api._generate.engine_id:
            results = api.generate(
                [prompt], [1.0], 
                w, h, 
                seed = seed,
                cfg_scale = cfg_scale, 
                init_image = image,
                init_strength = init_strength,
                mask = mask,
                masked_area_init = masked_area_init,
                mask_fixup = mask_fixup,
            )
        else:
            results = api.inpaint(
                image,
                mask,
                [prompt], [1.0], 
                seed = seed,
                cfg_scale = cfg_scale, 
                init_strength = init_strength,
                masked_area_init = masked_area_init,
                mask_fixup = mask_fixup,
            )
        result = results[generation.ARTIFACT_IMAGE][0]
        return utils.cv2_to_pil(result)
    elif mode == 'init':
        return utils.cv2_to_pil(image)
    else: # mode == 'mask'
        return utils.cv2_to_pil(mask)

def load_std():
    return (api._generate.engine_id, "original", 0.2, False, 16, 0, 16)

def load_inp():
    return (api._inpaint.engine_id, "zero", 0, False, 0, 0, 0)

with gr.Blocks() as demo:
    gr.Markdown("Stable Diffusion Inpainting")


    with gr.Row():
        with gr.Column():
            with gr.Row():
                input_rgb = gr.Image(label="image", type='pil')
                input_mask = gr.Image(label="mask", type='pil')
            with gr.Row():                
                input_prompt = gr.Text(label="prompt", value="spiderman selfie in chicago with city skyline in background", interactive=True)
                input_seed = gr.Number(label="seed", value=1337, precision=0, interactive=True)
                input_cfg = gr.Number(label="cfg_scale", value=7, interactive=True)
            with gr.Row():
                button_preset_std = gr.Button("Load standard settings")
                button_preset_inp = gr.Button("Load inpaint settings")
            with gr.Row():
                input_model = gr.Dropdown(label="model", choices=[api._generate.engine_id, api._inpaint.engine_id], value=api._inpaint.engine_id, interactive=True)
                input_mask_init = gr.Dropdown(label="mask init", choices=["original", "random", "zero"], value="zero", interactive=True)
                input_mode = gr.Dropdown(label="output", choices=["result", "init", "mask"], value="result", interactive=True)
            with gr.Row():
                input_strength = gr.Number(label="init strength", value=0.0, interactive=True)
                input_shuffle = gr.Number(label="shuffle", value=0.0, interactive=True, help="helps standard model to introduce more details")
                input_mask_erode = gr.Number(label="mask erode", value=0, precision=0, interactive=True)
                input_mask_blur = gr.Number(label="mask blur", value=0, precision=0, interactive=True)
                input_fixup = gr.Checkbox(label="mask fixup", value=False, interactive=True)
        with gr.Column():
            out_image = gr.Image(label="result")
            button_inpaint = gr.Button("Inpaint!")

    all_settings = [input_model, input_mask_init, input_strength, input_fixup, input_shuffle, input_mask_erode, input_mask_blur]

    button_preset_std.click(load_std, outputs=all_settings)
    button_preset_inp.click(load_inp, outputs=all_settings)

    button_inpaint.click(
        inpaint,
        inputs=[input_model, input_prompt, input_seed, input_cfg, input_strength, 
                input_fixup, input_mask_init, input_rgb, input_mask, input_shuffle, 
                input_mask_erode, input_mask_blur, input_mode],
        outputs=[out_image]
    )

demo.launch(show_api=False, debug=True, inline=True, height=1024, share=False)
